In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
import tabulate
from IPython.display import HTML, display, Markdown
import csv
import unicodedata
import getopt
from ipywidgets import HBox, VBox, Layout

import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args


def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_8D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))

#map powerplant categories
map_plants = dict()
map_plants['COAL'] = 'Coal'
map_plants['GEO'] = 'Geothermal'
map_plants['BIO'] = 'Biomass'
map_plants['HYD'] = 'Hydro'
map_plants['NG'] = 'Natural Gas'
map_plants['WND'] = 'Wind'
map_plants['SOL'] = 'Solar'
map_plants['URN'] = 'Nuclear'
map_plants['Batt'] = 'Battery'

col_order = ['Biomass','Geothermal','Hydro','Nuclear','Coal','Natural Gas','Solar','Wind','Hydrogen','Battery']

### Electric Sector Overview <a class="anchor" id="electric"></a>
The electric sector includes a representation of existing and new generation technologies, indexed by the region in which they are located. Thermal power plants include coal-fired steam, natural gas steam plants, open cycle and combined-cycle natural gas turbines with and without CCS, and light water nuclear reactors. Renewable sources include conventional hydro, centralized solar photovoltaics (PV), wind, biomass, and geothermal technologies.

Data for the electric sector is  compiled using <a href="https://github.com/gschivley/PowerGenome" rel="nofollow">PowerGenome</a>, an open-source tool that allows users to create input datasets for power system capacity expansion models. PowerGenome primarily uses data from the National Renewable Energy Laboratory (NREL), the US Energy Information Administration (EIA) and the US Environmental Protection Agency (EPA). Much of these underlying data have been compiled by the Catalyst Cooperative's Public Utility Data Liberation Project <a href="https://github.com/catalyst-cooperative/pudl" rel="nofollow">(PUDL)</a> into a single SQLite database that PowerGenome uses.

Using [PowerGenome](https://github.com/gschivley/PowerGenome), we experimented with the aggregation of balancing authorities (as defined by [EPA's Integrated Planning Model regions](https://www.epa.gov/sites/production/files/2019-03/documents/chapter_3_0.pdf)) into nine OEO regions, to develop the spatial representation for the electric sector shown in Figure 1. For all other sectors, we follow the spatial aggregation as shown in Figure 1 of the Overview notebook.

<img src="documentation_images/IPM_regions_to_temoa_state_groups.jpg" width="60%"/>

**Figure 1.** The nine US regions developed for the electric sector in the OEO input database, based on aggregations of IPM regions

#### Generator clusters <a class="anchor" id="gens_cluster"></a>
The following table presents an exhaustive list of all existing and new generation technologies included in the OEO input database.

In [2]:
df = pd.read_sql("SELECT * FROM technologies WHERE sector='electric' ORDER BY tech", con)
df['tech_desc'] = df['tech_desc'].str.replace('#','')
headers=["Technology","Description"]
display(Markdown('**Table 1.** Electric generators in the OEO input database'))
display(HTML(tabulate.tabulate(df[['tech','tech_desc']].set_index('tech'), headers, stralign='left',tablefmt='html')))

**Table 1.** Electric generators in the OEO input database

Technology,Description
E_BECCS_N,bio-energy with carbon capture and storage
E_BECCS_N_emissions,linked process that produces emissions associated with BECCS
E_BIO_N,new bio-energy
E_BIO_R,existing bio-energy
E_Batt,battery storage
E_Batt8hr,battery storage
E_COALIGCC_N,new coal IGCC power plant
E_COALSTM_R_1,new pulverized coal steam power plant
E_COALSTM_R_1_2025,existing coal steam plant
E_COALSTM_R_1_2030,existing coal steam plant


##### Existing generators <a class="anchor" id="exist_gens"></a>

Existing electricity sector operations in the modeled regions cannot be represented by individual generator operations due to computational issues. In order to develop a reduced-order representation of these generators that is tractably solved, [PowerGenome](https://github.com/gschivley/PowerGenome) uses k-means clustering techniques to aggregate existing generators into groups or clusters. The generators are grouped into clusters using four generator characteristics: nameplate capacity, heat rate, installation year, and fixed O&M costs.

In each region, existing conventional coal steam and natural gas combined cycle (NGCC) plants are represented by 4 clusters each, natural gas combustion turbine (NGCT), natural gas steam turbine (NGST), nuclear, and conventional hydroelectric plants are represented by 2 clusters each, while all other technology types - biomass, geothermal, centralized solar photovoltaic, onshore wind - are represented by a single cluster each.

##### New thermal generators <a class="anchor" id="new_gens"></a>

New thermal generators are represented by a single cluster for every modeled year and include NGCC, NGCT, NGCC with 90% efficient carbon capture and storage (CCS), NGCC with 100% efficient CCS, geothermal (hydro binary and hydro flash technologies), coal integrated gasification combined cycle (IGCC), ultra-supercritical pulverized coal with 90% efficient CCS, biomass combined-cycle, and hydrogen combined-cycle. Data for these technologies are derived from the NREL Annual Technology Baseline (ATB) via PowerGenome, except for a few listed below: 
* *Hydrogen combined-cycle turbines (E_H2CC_N)*: Hydrogen at 100 bar pressure can be burned in combined-cycle turbines to produce electricity, which is assumed to have the same techno-economic characteristics as NGCC generators, but without any combustion emissions.
* *Bio-energy with carbon capture and storage (E_BECCS_N)*: This technology representation comes from the original US EPA MARKAL database, and will be updated in the near-term.
* *Hydrogen storage (H2_STO150)*: This technology representation comes from [Dowling et al, 2020](https://www.cell.com/joule/pdfExtended/S2542-4351(20)30325-1).

In the near future, we plan to consider adding modular nuclear reactor technologies to the OEO input database as well, although this technology pathway is currently not reported in the NREL ATB.

##### New renewable generator clusters <a class="anchor" id="new_vre_gens"></a>
In all model regions, new utility solar PV and onshore wind capacities are represented by three clusters each, while offshore wind is represented by a single cluster in the CA, NW, NE regions. Techno-economic data for all renewable clusters of a single technology type (except for capacity factor and maximum available capacity) are identical and derived from the NREL ATB via PowerGenome. Further, PowerGenome uses k-means clustering to develop these groups, which differ by capacity factor and maximum available capacity in each region. This is explained in greater detail in [Section 1.5](#capfac).

Although PowerGenome doesn't explicitly develop technology clusters for new residential-scale PV, we can use the tool to develop cost estimates (investment and O\&M costs) for a single representative cluster, with underlying data from the NREL ATB. As an approximation, we use the capacity factors developed for the three utility PV clusters in each region to represent residential PV generation as well. Each cluster has the same cost estimates, as developed through PowerGenome. Residential PV annual generation is specified exogenously, with data from the [NREL dGen](https://maps.nrel.gov/dgen) model's Mid (PV cost) scenario - data available at the state level is aggregated to each of the nine OEO regions. This technology is assumed to have no other resource constraints.
 
We also develop a single cluster of concentrating solar thermal technologies in the California and Southwestern US regions. In the NREL ATB, the [representative](https://atb.nrel.gov/electricity/2020/index.php?t=sc) technology is assumed to be a 100 MW molten salt power-tower with 10 hours of thermal energy storage, driven by Class 5 (excellent) resources. While NREL reports an average capacity factor of 64% for this technology pathway, we currently do not have any hourly capacity factor information available, and therefore we assume a constant 64% capacity factor across all time periods for now. 

The sections that follow provide details for the data used to generate the input database for the OEO.


#### Costs <a class="anchor" id="costs"></a>
Investment and operating costs of electric generators are drawn from the 2021 NREL ATB via PowerGenome. All costs are reported in 2018US$.

While the ATB publishes data for a number of relevant scenarios, the following parameters are used to select data for the OEO input database.

|Parameter | Value | Notes |
| :----- | :-- |:--------- |
|ATB Base Year | 2019 | |
|Technology costs case | Moderate | with "expected level of technology innovation" <a href = "https://atb.nrel.gov/electricity/2021/definitions" rel="nofollow">(NREL, 2021)|
|Financial case| Market | "includes effects of tax reform and tax credits"  <br>  <a href="https://atb.nrel.gov/electricity/2021/definitions" rel="nofollow">(NREL, 2021) |
|Capital recovery period | 20 years | |
|AC to DC ratio for PV | 1.3 | utility-scale solar PV is assumed to be single axis tracking |

##### Investment costs <a class="anchor" id="costinvest"></a>
Overnight capital costs from the NREL ATB are used to characterize the investment costs of technologies that will likely be available in the modeling time-horizon, for all technologies other than the few mentioned in [Section 1.1.2](#new_gens). All technologies other than solar and wind are limited are represented as a single technology cluster. For solar and wind technologies, multiple clusters that have varying capacity factors and maximum available capacity are developed using the clustering techniques within PowerGenome. However, all clusters of a single technology type are assumed to have the same investment costs.

The ATB does not report variations in investment costs across regions, and so PowerGenome uses appropriate regional cost-multipliers from the EIA Annual Energy Outlook (AEO) Electricity Market Module (EMM) in order to adjust capital costs. According to the EIA [EMM documentation](https://www.eia.gov/outlooks/aeo/assumptions/pdf/electricity.pdf), these multipliers can account for regional variations in factors such as ambient air conditions and how that may affect combustion turbine available capacity, or accessing wind resources. These multipliers are not available for all technology-region combinations, and so the following approximations are made:

* *Bio-energy with carbon capture and storage (BECCS)*: Regional multipliers assumed to be the same as coal IGCC-CCS plants.
* *Hydrogen storage (H2_STO150)*: Regional multipliers assumed to be the same as battery technologies.
* *Geothermal (E_GEOF_N, E_GEOB_N)*: Regional multipliers assumed to be the same as NGCC plants.
* *Any remaining technologies within a region that do not have cost multiplier data*: Regional multipliers assumed to be the average of multipliers for all available technologies within that region

The form below can be used to display the investment costs by technology, region, and time period.


In [3]:
def show_cost_invest(con):

    df = pd.read_sql(
        "SELECT regions, tech,vintage, cost_invest, cost_invest_units FROM CostInvest WHERE tech IN (SELECT tech FROM technologies WHERE sector=='electric') ORDER BY tech, vintage",
        con)

    display_types = ['table', 'figure']
    
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()


    def filter_tech(tech='', region='', o_format=''):
        if tech == 'All':
            df_sel = df[(df.regions == region)]
        else:
            df_sel = df[(df.tech.str.contains(tech)) & (df.regions == region)]
        if o_format == 'table':
            df_sel = df_sel.pivot_table(
                index=['regions', 'tech', 'cost_invest_units'],
                columns='vintage',
                values='cost_invest').reset_index().set_index('regions')
            df_sel.rename(columns={'cost_invest_units': 'units'}, inplace=True)
            display(
                HTML(
                    tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".0f",
                                      tablefmt='html')))
        elif o_format == 'figure':
            fig, ax = plt.subplots(figsize=(10, 6))
            for ind_tech in df_sel.tech.unique():
                plt.plot(df_sel[df_sel.tech == ind_tech].vintage,
                         df_sel[df_sel.tech == ind_tech].cost_invest,
                         label=ind_tech)
            plt.legend()
            plt.ylabel('Investment costs ($M/GW)')
            plt.ylim([0, df.cost_invest.max() * 1.1])
            plt.xlabel('Vintage')


    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_invest(con)

##### Fixed Operations and Maintenance (O&M) Costs <a class="anchor" id="costFOM"></a>
New generator costs are derived from the NREL ATB via PowerGenome. Solar PV technology fixed costs reported by NREL on a DC basis are adjusted using an assumed AC to DC ratio for PV of 1.3 (as noted in the table above). PowerGenome uses data from the [EIA](https://www.eia.gov/analysis/studies/powerplants/generationcost/pdf/full_report.pdf) to characterize fixed O&M costs associated with existing individual generators, where available. Mean costs based on available data are estimated for categories of generators based on their capacity (e.g., for combined cycle plants between 500 and 1000 MW) and assigned to all the generators in that category. For generator types where these data are not available from EIA, PowerGenome uses data for new generators from the NREL ATB and linearly adjusts them to existing generators based on their respective heat rate values.

In [4]:
def show_cost_fixed(con):
    df = pd.read_sql("SELECT regions, tech,vintage, cost_fixed_units, AVG(cost_fixed) AS cost_fixed FROM CostFixed WHERE tech IN (SELECT tech FROM technologies WHERE sector=='electric') GROUP BY regions, tech,vintage ORDER BY tech, vintage", con)
    display_types = ['table', 'figure']
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))    
    regions = [x for x in df.regions.unique() if '-' not in x]
    def filter_tech(tech ='', region = '', o_format=''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)]
        if o_format=='table':
            #df_sel = df_sel.pivot_table(index=['regions','tech', 'cost_fixed_units'], columns='vintage', values='cost_fixed').reset_index().set_index('regions')
            header = ['regions', 'technology', 'vintage', 'units', 'fixed cost']
            display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header,  floatfmt=".0f" , tablefmt='html')))
        elif o_format=='figure':
            fig, ax = plt.subplots(figsize=(10,6))
            for ind_tech in df_sel.tech.unique():
                plt.bar(df_sel[df_sel.tech==ind_tech].vintage, df_sel[df_sel.tech==ind_tech].cost_fixed, label=ind_tech)
            plt.legend()
            if len(df_sel)==1:
                plt.xlim([df_sel.vintage.values - 2, df_sel.vintage.values + 2])
                plt.xticks(np.arange(df_sel.vintage.values - 1, df_sel.vintage.values + 2))
            plt.ylabel('Fixed costs ($M/GWyr)')
            #plt.ylim([0, df.cost_fixed.max()*1.1])
            plt.xlabel('Vintage')

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)
    
show_cost_fixed(con)

##### Variable Operations and Maintenance Costs <a class="anchor" id="costVOM"></a>
New generator costs are derived from the NREL ATB via PowerGenome. PowerGenome uses data from the [EIA](https://www.eia.gov/analysis/studies/powerplants/generationcost/pdf/full_report.pdf) to characterize variable costs of existing individual generators, where available. Mean costs based on available data are estimated for categories of generators based on their capacity (e.g., for combined cycle plants between 500 and 1000 MW) and assigned to all the generators in that category. For generator types where these data are not available from EIA, PowerGenome uses data for new generators from the NREL ATB and linearly adjusts them to existing generators based on their respective heat rate values.

In [5]:
def show_cost_variable(con):

    df = pd.read_sql("SELECT regions, tech,vintage, cost_variable_units, AVG(cost_variable) AS cost_variable FROM CostVariable WHERE tech IN (SELECT tech FROM technologies WHERE sector=='electric') AND cost_variable> 0 GROUP BY regions, tech,vintage ORDER BY vintage", con)
    display_types = ['table', 'figure']
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech ='', region = '', o_format=''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)]
        if o_format=='table':
            #df_sel = df_sel.pivot_table(index=['regions','tech', 'cost_variable_units'], columns='vintage', values='cost_variable').reset_index().set_index('regions')
            header = ['regions', 'technology', 'vintage', 'units', 'variable cost']
            display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header,  floatfmt=".2f" , tablefmt='html')))
        elif o_format=='figure':
            fig, ax = plt.subplots(figsize=(10,6))
            for ind_tech in df_sel.tech.unique():
                plt.bar(df_sel[df_sel.tech==ind_tech].vintage, df_sel[df_sel.tech==ind_tech].cost_variable, label=ind_tech)
            plt.legend()
            if len(df_sel)==1:
                plt.xlim([df_sel.vintage.values - 2, df_sel.vintage.values + 2])
                plt.xticks(np.arange(df_sel.vintage.values - 1, df_sel.vintage.values + 2))
            plt.ylabel('Variable costs ($M/PJ)')
            #plt.ylim([0, df.cost_variable.max()*1.1])
            plt.xlabel('Vintage')
            ax.xaxis.set_major_locator(MaxNLocator(integer=True))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w3 = widgets.Select(options=display_types)
    w = widgets.interactive(filter_tech, tech=w1, region=w2, o_format=w3)

    controls_rows(w)

show_cost_variable(con)

#### Existing Capacity <a class="anchor" id="exist_cap"></a>
PowerGenome uses generator data from 2018 to report the existing electric generation capacity. The (winter) capacities for individual generators are obtained from the US EIA 860 and 923 files via the [PUDL](https://github.com/catalyst-cooperative/pudl) database. PowerGenome then combines the individual generators to create a few clusters that are representative of the entire fleet, using k-means clustering.

In [6]:
def show_exist_cap(con, map_plants):
    df = pd.read_sql("SELECT regions, tech,vintage, exist_cap FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies \
    WHERE sector=='electric' AND flag!='ps') AND tech!='E_TRANS_R' ORDER BY tech", con)
    df.loc[:,'agg_tech'] = [map_plants[y] for x in df.tech for y in map_plants.keys() if y.lower() in x.lower()] #map agg technologies
    df_sum = df.drop("vintage", axis=1).groupby(by = ['regions','agg_tech','tech']).sum().reset_index()
    df_sum.sort_values(by='exist_cap', ascending=False, inplace=True)
    techs = ['All'] + list(df_sum.agg_tech.unique())
    regions = df_sum.regions.unique()
    def filter_tech(tech ='', region = ''):
        if tech=='All':
            df_sel = df_sum[(df_sum.regions==region)]
        else:
            df_sel = df_sum[(df_sum.agg_tech==tech) & (df_sum.regions==region)]
        df_sel = df_sel.sort_values(by='tech')
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), ["regions", "technology type","technology", "capacity (GW)" ], floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_exist_cap(con, map_plants)


#### Efficiency <a class="anchor" id="efficiency"></a>
The efficiency of each generator cluster is represented as the mean of the individual generators in each cluster. PowerGenome compiles existing generator heat rates from the US EIA 860 and 923 files via the [PUDL](https://github.com/catalyst-cooperative/pudl) database, and heat rates for new technologies from the NREL ATB. The total capacity of each technology cluster and the associated efficiencies are reported in the table below.

In [7]:
def show_efficiency(con):
    df = pd.read_sql("SELECT regions, tech, vintage, efficiency FROM Efficiency WHERE tech IN (SELECT tech FROM technologies WHERE sector=='electric') AND efficiency < 1 ORDER BY tech, vintage", con)
    df_cap = pd.read_sql("SELECT regions, tech,vintage, exist_cap FROM ExistingCapacity WHERE tech IN (SELECT tech FROM technologies WHERE sector=='electric') ORDER BY tech, vintage", con)
    df_cap.exist_cap = np.round(df_cap.exist_cap*100)/100
    df = pd.merge(df, df_cap, on = ['regions','tech','vintage'], how='left')
    df.fillna('--', inplace=True)
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    regions = [x for x in regions if '-' not in x]
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)].copy()
        else:
            df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)].copy()
        df_sel.efficiency = 100*df_sel.efficiency
        header = ['region','technology','vintage','efficiency (%)','capacity (GW)']
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, floatfmt=".1f", tablefmt='html')))

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_efficiency(con)

#### Capacity Factors and Capacity of Variable Renewable Energy <a class="anchor" id="capfac"></a>

A number of clusters representing variable renewable technologies (as presented in [Section 1.1.3](#new_vre_gens)) are selected for each region using clustering techniques in PowerGenome. 

Annual hourly electricity load and variable renewables capacity factor data for a representative year (2012) are used. Historical capacity factors of existing capacity are obtained from EIA, while data for potential new generators are obtained from datasets developed by <a href="https://www.vibrantcleanenergy.com/products/" rel="nofollow">Vibrant Clean Energy</a>. The renewables capacity factors for potential new generators are modeled using 2012 weather data. VRE capacity factor and maximum available resource data are available through these datasets at a specific grid-level resolution for the entire United States. These are aggregated to the specified number of technology clusters in each of the nine OEO regions using clustering techniques. Further details on these datasets are forthcoming (in PowerGenome documentation).

In the current OEO database, annual energy supply and demand is represented by a few days at an hourly resolution. Specifically, service demands across all the sectors as well as variable renewable energy supply are characterized at this resolution for all the US modeled regions. Therefore the annual hourly datasets for each specified technology cluster are jointly downscaled to a few representative days, using k-means clustering within PowerGenome (e.g., see [Mallapragada et al, 2018](https://doi.org/10.1016/j.energy.2018.08.015)).

In [8]:
def show_capfac(con):

    df = pd.read_sql("SELECT regions, season_name,time_of_day_name,tech,cf_tech FROM CapacityFactorTech ORDER BY tech", con)
    df_maxcap = pd.read_sql("SELECT regions,tech, periods, maxcap FROM MaxCapacity WHERE tech in (SELECT DISTINCT(tech) FROM CapacityFactorTech)", con)
    df_maxcap = df_maxcap.groupby(by=['regions','tech']).mean().reset_index().drop(columns=['periods'])
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)]
        header = ['region','season','time-of-day','technology','capacity factor']
        fig, ax = plt.subplots(figsize=(10,6))
        for ind_tech in df_sel.tech.unique():
            plt.plot(np.arange(len(df_sel[df_sel.tech==ind_tech])), df_sel[df_sel.tech==ind_tech].cf_tech, label= ind_tech)
        plt.legend()
        plt.ylabel('Capacity Factor')
        plt.ylim([0,1])
        plt.xlabel('Hour')
        plt.title('Power plant capacity factors by technology cluster')
        
        df_sel_maxcap = df_maxcap[(df_maxcap.tech.str.contains(tech)) & (df_maxcap.regions==region)]
        
        header = ['region','technology','maximum capacity (GW)']
        display(HTML(tabulate.tabulate(df_sel_maxcap.set_index('regions'), header, floatfmt=".0f", tablefmt='html')))
       

    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)

show_capfac(con)

As noted earlier, annual residential PV generation is exogenous and drawn from [NREL dGen](https://maps.nrel.gov/dgen) scenario results. These values are presented in the table that follows.

In [9]:
df = pd.read_sql("SELECT * FROM MinGenGroupTarget", con)
df['min_act_g'] /= (0.0036*1000)
df_sel = df.pivot_table(
                index=['group_name'],
                columns='periods',
                values='min_act_g').reset_index().set_index('group_name')
df_sel.index = df_sel.index.str.replace('_RESPVGRP','')

display(Markdown('**Annual residential PV generation, from the NREL dGen model (TWh)**'))
display(HTML(tabulate.tabulate(df_sel,
                                      ['region'] + list(df_sel.columns.values),
                                      floatfmt=".1f",
                                      tablefmt='html')))

**Annual residential PV generation, from the NREL dGen model (TWh)**

#### Capacity Credit <a class="anchor" id="capcredit"></a>
Current values assume a default capacity credit value of 0.9 for all dispatchable generators, and 0.15 for all other generators, across the modeling time-horizon.

Specifying the capacity credit for variable renewables like wind and solar is challenging because the credit declines as a function of installed capacity within the system. Trying to endogenously update the capacity credit within a single model solve requires either a non-linear or mixed integer formulation of Temoa, which greatly enhances the computational burden. However, since the Temoa OEO database is being solved myopically, we are developing the capability to update the capacity credit in-between solves, similar to the ReEDS model.

Once a single time period solve is completed, the optimized electric generator capacities and electricity demand by region are passed to a module that updates the capacity credit for wind and solar by region, which will be used in the next time period solve. The code works by performing a simulation that compares available capacity to annual demand, as explained below.

To calculate available capacity, each existing generating unit plus the new installed capacity are assigned an Equivalent Forced Outage Rate - Demand (EFORd) value. The EFORd value represents the probability that the generator will be unavailable due to a forced outage or derating during a time when there is demand for this generator. Random numbers with a [0,1] range are drawn for each existing generating unit and new dispatchable generating capacity and compared with its EFORd value. Note that Temoa optimizes new capacity in aggregate by technology type rather than individual units. Thus, for the purposes of this calculation, new capacity is split into hypothetical units, where the capacity of each unit represents the average for that technology type.  If the random number exceeds the EFORd value, the generator is unavailable to meet demand; else, it is available. The exercise is repeated 50,000 times to obtain a large set of available generation capacity estimates by region.

To calculate net demand, the product of installed wind and solar capacities multiplied by their respective hourly capacity factors are subtracted sequentially from the regional demand returned by Temoa. Since different clusters of wind and solar within Temoa have different capacity factors, the calculation takes into account the differences across clusters. (Note that the amount of electricity demanded by Temoa in each time period and region is determined endogenously, based on the relative cost-effectiveness of fuels and technologies used to meet end-use service demands.)

Using this data, a standard reliability analysis is performed. For each hour of estimated net demand, the code compares all 50,000 available capacity estimates and records any instances where the net demand exceeds the available capacity. When this count is divided by the total number of iterations, it yields a loss of load probability (LOLP) estimate. A uniform adjustment to the demand  in all hours is made to bring each region to its benchmark reliability (an LOLP of 1 day in 10 years). 

Once the load has been adjusted to meet the LOLP benchmark, the capacity credits associated with increments of solar and wind in the next model time period are calculated. To do so, the procedure described above is repeated with an additional 1000 MW of wind or solar capacity. Once again, this new capacity increment is multiplied by the hourly capacity factor and subtracted from load to create a new net load profile. Since wind and solar work to reduce demand, their addition to the system may reduce the calculated LOLP. The demand increment that brings the LOLP back to its benchmark value -- known as the effective load carrying capability (ELCC) -- is determined. The ELCC represents the absolute amount of capacity credit that the installed wind and solar in each region should receive. The ELCC divided by the 1000 MW regional solar (or wind) capacity addition yields the capacity credit as a fraction. The calculation is repeated for each cluster of wind and solar (i.e., each resource profile) in each region, so that they each have their own unique capacity credit estimate for the next solve.

This whole procedure will be repeated after each time period solve, until the end of the model time horizon is reached.

In [10]:
def show_capacity_credit(con):

    df = pd.read_sql("SELECT regions, periods, tech, cf_tech FROM CapacityCredit", con)
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)]
        df_sel = df_sel.pivot_table(index=['regions','tech'], columns='periods', values='cf_tech').reset_index().set_index('regions')
        header = ['region'] + list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel, header, tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_capacity_credit(con)

#### Ramp rates <a class="anchor" id="ramp_rates"></a>
PowerGenome uses ramp rates from a [study by the California Utilities Public Commission](https://www.cpuc.ca.gov/General.aspx?id=6442462824). All clusters of a single technology, across any region, are assumed to have the same ramp rate.

In [11]:
def show_ramp_rate(con):
    df = pd.read_sql("SELECT regions, tech, ramp_up FROM RampUp", con)
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)]
        df_sel['ramp_up'] = df_sel['ramp_up']*100
        header = list(['region', 'technology', 'ramp rate (% per hour)'])
        display(HTML(tabulate.tabulate(df_sel.set_index('regions'), header, tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_ramp_rate(con)

#### Discount rates <a class="anchor" id="disc_rates"></a>
Discount rates for individual new generating technologies are represented by the weighted average cost of capital (WACC) in the NREL ATB, which is the average rate paid to finance the installations. These rates do not vary by region.

In [12]:
def show_disc_rate(con):
    df = pd.read_sql("SELECT regions, tech, vintage, tech_rate FROM DiscountRate \
    WHERE tech in (SELECT tech FROM technologies WHERE sector='electric')", con)
    techs = []
    for unique_tech in df.tech.unique():
        try:
            int(unique_tech.split('_')[-1])
            techs.append('_'.join(unique_tech.split('_')[0:-1]))
        except:
            techs.append(unique_tech)
    techs = ['All'] + list(set(techs))
    regions = df.regions.unique()
    def filter_tech(tech='', region =''):
        if tech=='All':
            df_sel = df[(df.regions==region)]
        else:
            df_sel = df[(df.tech.str.contains(tech)) & (df.regions==region)]        
        df_sel = df_sel.pivot_table(index=['regions','tech'], columns='vintage', values='tech_rate').reset_index()
        header = list(df_sel.columns.values)
        display(HTML(tabulate.tabulate(df_sel, header, floatfmt=".3f", tablefmt='html')))
    w1 = widgets.Select(options=techs)
    w2 = widgets.Select(options=regions)
    w = widgets.interactive(filter_tech, tech=w1, region=w2)

    controls_rows(w)
    
show_disc_rate(con)

#### Transmission <a class="anchor" id="transmission"></a>
Transmission data from PowerGenome comes from EPA's bulk transmission data for IPM regions. Existing transmission data primarily include line losses (as reported in the Efficiency table), and maximum line capacity. For a pair of regions, the maximum line capacity may differ, depending on the direction of transmission. For our modeling purposes, we choose the maximum of values in both directions for a pair of regions, as a maximum line capacity value for that pair.

##### Existing transmission <a class="anchor" id="linecap"></a>
Capacity and losses associated with existing inter-regional tranmission are presented in the table below, based on data from PowerGenome. These transmission lines are assumed to have no operating costs. However, intra-region operating costs and losses are assigned separately, based on data from the US EIA Annual Energy Outlook (AEO). 

In [13]:
def show_trans_cap_efficiency(con):
    df = pd.read_sql("SELECT C.regions, C.exist_cap, C.exist_cap_units, \
    E.efficiency \
    FROM ExistingCapacity AS C \
    INNER JOIN Efficiency AS E ON \
    C.regions = E.regions AND \
    C.vintage = E.vintage AND \
    C.tech = E.tech \
    WHERE C.tech LIKE '%E_TRANS_R%'", con)


    #df = df.pivot_table(index=['regions','tech','exist_cap_units'], columns='vintage', values='exist_cap').reset_index()
    df.loc[:,'region 1'] = df.regions.str.split('-').apply(lambda x: x[0])
    df.loc[:,'region 2'] = df.regions.str.split('-').apply(lambda x: x[1])
    df.loc[:, 'efficiency'] = (1 - df.loc[:, 'efficiency'])*100
    header = ['region 1', 'region 2', 'existing capacity (' + df['exist_cap_units'].unique()[0] + ')', 'line loss (%)']
    df_sel = df.drop(['exist_cap_units','regions'], axis=1).set_index(['region 1', 'region 2']).reset_index().set_index('region 1')

    display(HTML(tabulate.tabulate(df_sel, header, floatfmt=".1f", tablefmt='html')))

show_trans_cap_efficiency(con)

region 1,region 2,existing capacity (GW),line loss (%)
CA,NW,6.8,6.2
CA,SW,12.0,4.9
NW,SW,4.5,5.8
SW,CEN,0.6,7.3
TX,CEN,2.5,4.9
N_CEN,CEN,15.4,4.9
N_CEN,SE,6.1,8.9
N_CEN,MID_AT,16.6,7.5
CEN,SE,8.1,7.8
SE,MID_AT,10.1,4.9


Intra-region T\&D losses are assumed to be 4.7\% based on the <a href="https://www.eia.gov/outlooks/aeo/data/browser/#/?id=8-AEO2018&cases=ref2018&sourcekey=0" rel="nofollow">2019 AEO</a>, calculated using the ratio of the net generation to the grid and the total net electricity generation for the years 2019-2020. Intra-region T&D operating costs were based on projections from the <a href="https://www.eia.gov/outlooks/aeo/data/browser/#/?id=8-AEO2019&region=0-0&cases=ref2019&start=2017&end=2050&f=A&linechart=ref2019-d111618a.75-8-AEO2019~ref2019-d111618a.76-8-AEO2019&map=&ctype=linechart&sourcekey=0" rel="nofollow">2019 AEO</a> and are presented in the table below. 

In [ ]:
def t_d_variableom(con):
    df = pd.read_sql("SELECT DISTINCT periods, cost_variable FROM CostVariable WHERE tech = 'E_ELCTDLOSS'", con)

    header = ['period', 'intra-region T&D O&M ($M/PJ)']
    display(HTML(tabulate.tabulate(df.set_index('periods'), header, floatfmt=".1f", tablefmt='html')))

t_d_variableom(con)

##### New transmission <a class="anchor" id="trans_FOM"></a>
New transmission line investment costs are obtained from PowerGenome. The underlying transmission line cost per MW-mile data for each region comes from the [NREL ReEDS model](https://www.nrel.gov/docs/fy17osti/67067.pdf), which PowerGenome uses to calculate costs per MW along with estimates of average distance between the regions. A discount rate of 6.9% and investment lifetime of 60 years is assumed, based on [Gorman et al, 2019](https://www.sciencedirect.com/science/article/pii/S0301421519305816). Transmission line losses for any new transmission capacity is assumed to be the same as existing capacity for each pair of regions. Intra-regional losses and variable O&M costs are assumed to be the same as that for existing transmission.

In [14]:
def show_trans_costs(con):
    df = pd.read_sql("SELECT regions, vintage, tech, cost_invest, cost_invest_units FROM CostInvest WHERE tech LIKE '%E_TRANS_N%'", con)
    df = df.pivot_table(index=['regions','tech', 'cost_invest_units'], columns='vintage', values='cost_invest').reset_index()
    df.loc[:,'region 1'] = df.regions.str.split('-').apply(lambda x: x[0])
    df.loc[:,'region 2'] = df.regions.str.split('-').apply(lambda x: x[1])
    df_sel = df.drop(['tech','regions'], axis=1).set_index(['region 1', 'region 2']).reset_index().set_index('region 1')
    df_sel.rename(columns={'cost_invest_units': 'units'}, inplace=True)
    display(HTML(tabulate.tabulate(df_sel, ['region 1'] + list(df_sel.columns.values), floatfmt=".0f", tablefmt='html')))

show_trans_costs(con)

region 1,region 2,units,2020,2025,2030,2035,2040,2045,2050
CA,NW,$M/GW,1470,1470,1470,1470,1470,1470,1470
CA,SW,$M/GW,1158,1158,1158,1158,1158,1158,1158
CEN,N_CEN,$M/GW,693,693,693,693,693,693,693
CEN,SE,$M/GW,1177,1177,1177,1177,1177,1177,1177
CEN,SW,$M/GW,1222,1222,1222,1222,1222,1222,1222
CEN,TX,$M/GW,826,826,826,826,826,826,826
MID_AT,NE,$M/GW,1453,1453,1453,1453,1453,1453,1453
MID_AT,N_CEN,$M/GW,1072,1072,1072,1072,1072,1072,1072
MID_AT,SE,$M/GW,741,741,741,741,741,741,741
NE,MID_AT,$M/GW,1453,1453,1453,1453,1453,1453,1453


#### Technology/commodity description lookup tool <a class="anchor" id="description_look_up"></a>
Use the tool below to search for any key words that may describe a technology or commodity of interest (e.g. solar thermal). The tool provides a list of all the commodities and technologies in the database that may be relevant to the query.

In [15]:
w = widgets.Text(value='solar thermal')
display(w)
def f(w):
    if len(w)>0:
        df1 = pd.read_sql("SELECT * FROM commodities WHERE comm_desc LIKE '%" + w + "%'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech_desc LIKE '%" + w + "%'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'


        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])
        
        if len(df)>0:
            display(HTML(tabulate.tabulate(df.set_index('comm_tech'),['technology/commodity','type','description'],tablefmt='html')))
        else:
            print('')
    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='solar thermal')

Output()

#### Network diagram lookup tool  <a class="anchor" id="network_look_up"></a>
Use the [description lookup tool](#description_look_up) above to identify specific commodity or technology names. Type the name in the box below to generate a corresponding network diagram for that commodity or technology. The slider can be used to view different upstream levels of the network diagram.

In [16]:
w = widgets.Text(value='ELCP_Renewables')
display(w)
def f(w):
    if len(w)>0:
        
        df1 = pd.read_sql("SELECT comm_name, comm_desc FROM commodities WHERE comm_name='" + w + "'", con)
        df1['desc'] = df1['comm_desc'].str.replace('#','').str.strip()
        df1['comm_tech'] = df1['comm_name']
        df1['type'] = 'commodity'

        df2 = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)
        df2['desc'] = df2['tech_desc'].str.replace('#','').str.strip()
        df2['comm_tech'] = df2['tech']
        df2['type'] = 'technology'

        df = pd.concat([df1[['comm_tech','type','desc']], df2[['comm_tech','type','desc']]])

    
        if len(df)>0:
            def show_desc(level):
                display(Markdown(df['desc'][0]))
                final_dem = df['comm_tech'][0]
                df_graph = return_flowd_table(final_dem,level)
                args = create_args_flowd(df_graph)
                colors, quick_run_dot_fmt = return_format_colors()
                args.update(colors)
                #o_str = 'rankdir = "LR" ;'
                #r_str = 'rankdir = "LR" ; \n\t size="8,8";'
                #quick_run_dot_fmt = quick_run_dot_fmt.replace(o_str, r_str)
                dot_graph = quick_run_dot_fmt % args
                display(graphviz.Source(dot_graph))
            w2 = widgets.IntSlider(value=1,min=0,max=10,step=1,description='Level:',disabled=False,continuous_update=True,orientation='horizontal',readout=True,readout_format='d')
            w = widgets.interactive(show_desc, level=w2)
            controls_rows(w)
        else:
            print('')

    else:
        print('')
            


out = widgets.interactive_output(f, {'w': w})
display(out)

Text(value='ELCP_Renewables')

Output()

#### Technology/commodity look-up tool <a class="anchor" id="lookup"></a>
Use the tool below to retrieve the description for any technology or commodity within the database. Type the commodity or technology name in the box below to view the description. Note that names are case sensitive.

In [17]:
w = widgets.Text(value='ELC')
display(w)
def f(w):
    df = pd.read_sql("SELECT * FROM commodities WHERE comm_name='" + w + "'", con)
    if len(df)==0:
        df = pd.read_sql("SELECT * FROM technologies WHERE tech='" + w + "'", con)

    if len(df)>0:
        try:
            display(Markdown((df['comm_desc'].values[0].replace('#', '').strip())))
        except:
            display(Markdown(df['tech_desc'].values[0].replace('#', '').strip()))
    else:
        print('')


out = widgets.interactive_output(f, {'w': w})
display(out)


Text(value='ELC')

Output()